In [1]:
#from Node import Control_node 
from Node.Node import Control_node

### Agent
Fields:
`Agent(inputs, behavioral_model, system_estimate, reference, reference_update, sensor_function, comparator_function, controller_update, internal_model_update)`
<br/>
**inputs** - source of inputs (i.e. environment, state observations, others behaviors)
<br/>
**behavioral model** - initial matrix expressing future state given last behavior ("B" matrix)
<br/>
**system estimate** - initial matrix expressing future state given last state ("A" matrix)
<br/>
**reference** - target state 
<br/>
**reference update** - update to reference (target) signal
<br/>
**sensor function** - for taking in (and processing) observations
<br/>
**comparator function** - for comparing observation and estimate
<br/>
**controller** - for generating output
<br/>
**controller update** - for updating the behavioral matrix 
<br/>
**internal model** - for generating forward prediction
<br/>
**internal model update** - for updating the system estimate 
<br/>


In [2]:
import numpy as np

In [3]:

se = np.array([[0.3,0,0],
               [0,0.3,0],
               [0,0,0.3]])
ps = np.array([0.3,0.1,0.1])
bm = np.array([[0.1,0,0],
               [0,0.7,0],
               [0,0,0.5]])
po = np.array([0.3,0.7,0.1])

def imcc(system_estimate, previous_state, behavioral_model, previous_output):
    # actual estimate
    state = (system_estimate @ previous_state) + (behavioral_model @ previous_output)
    return state

state = imcc(se,ps,bm,po)
print(state)

[0.12 0.52 0.08]


In [4]:
# behavioral matrix (maps previous behavior to future behavior commands - how should I behave?)
B1 = np.array([[0.4,0,0],
               [0,0.3,0],
               [0,0,0.2]])
B2 = np.array([[0.1,0,0],
               [0,0.7,0],
               [0,0,0.5]])
# state/system matrix (maps previous state to future state - how will others behave (give past behaviors)?)
A1 = np.array([[0.4,0,0],
               [0,0.4,0],
               [0,0,0.4]])
A2 = np.array([[0.3,0,0],
               [0,0.3,0],
               [0,0,0.3]])
# reference matrix (how do I expect/want others to behave? target values)
R1 = np.array([0.3,0.5,0.1])
R2 = np.array([0.1,0.4,0.8])

# get the state observation (behaviors of others)
def sense(inputs):
    return inputs

# contrast reference with observations (and potentially incorporate predictions from internal model)
def compare(sensory_signal, reference, prediction):
    error = reference - sensory_signal
    return error

# generate a behavior
def control(error, behavioral_model, previous_output):
    #learning_rate = 0.01
    output = behavioral_model @ previous_output
    return output


# adjusts behavioral model to get closer to behavior given 
def control_update(error, behavioral_model, previous_output):
    # LMS algorithm
    learning_rate = 0.01
    behavioral_model = behavioral_model + learning_rate * error * previous_output
    return behavioral_model

# generate predictions on next state
def imc(system_estimate, previous_state, behavioral_model, previous_output):
    # actual estimate
    state = (system_estimate @ previous_state) + (behavioral_model @ previous_output)
    return state

# adjust system estimate to infer observations given my behavior
def imc_update(inputs, system_estimate, previous_behavior, behavioral_model, previous_state, learning_rate):
    # actual estimate
    curr_system_est = (system_estimate @ previous_state) + (behavioral_model @ previous_behavior)
    error = curr_system_est - inputs
    #learning_rate = 0.01
    system_estimate = system_estimate + learning_rate * error * previous_behavior
    return system_estimate
    
# behavior = last observation * behavioral model (like a motor command u)
# output = your behavior * est

In [5]:
print(B1.shape[1])
#agent2 = Control_node()
agent1 = Control_node(sensor=sense, comparator=compare, control_update=control_update, controller = control, internal_model=imc, internal_model_update=imc_update, behavioral_model=B1, system_estimate=A1, reference=R1)
agent2 = Control_node(sensor=sense, comparator=compare, control_update=control_update,  controller = control, internal_model=imc, internal_model_update=imc_update, behavioral_model=B2, system_estimate=A2, reference=R2)


3


In [7]:
trials = 100
behavior1 = np.array([0,0,0])
behavior2 = np.array([0,0,0])
agent1_behaviors = []
agent2_behaviors = []
agent1_error = []
agent2_error = []

for t in range(trials):
    b1 = agent1.go(behavior2)
    b2 = agent2.go(behavior1)

    agent1_behaviors.append(agent1.get_output())
    agent2_behaviors.append(agent2.get_output())
    agent1_behaviors.append(agent1.get_error())
    agent2_behaviors.append(agent2.get_error())

    behavior1 = b1
    behavior2 = b2


previous output len:  3
control
output:  [0.409 0.309 0.209]
previous output:  [0.4 0.3 0.2]
update
previous output:  [0.4 0.3 0.2]
previous output len:  3
control
output:  [0.113 0.713 0.513]
previous output:  [0.1 0.7 0.5]
update
previous output:  [0.1 0.7 0.5]
previous output len:  3
control
output:  [0.16387 0.09387 0.04387]
previous output:  [0.409 0.309 0.209]
update
previous output:  [0.409 0.309 0.209]
previous output len:  3
control
output:  [0.02087 0.50087 0.26087]
previous output:  [0.113 0.713 0.513]
update
previous output:  [0.113 0.713 0.513]
previous output len:  3
control
output:  [0.16750614 0.09660614 0.04570614]
previous output:  [0.16387 0.09387 0.04387]
update
previous output:  [0.16387 0.09387 0.04387]
previous output len:  3
control
output:  [0.02440719 0.51220719 0.26960719]
previous output:  [0.02087 0.50087 0.26087]
update
previous output:  [0.02087 0.50087 0.26087]
previous output len:  3
control
output:  [0.06699652 0.02960952 0.01022252]
previous output:  